In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Lambda
from keras.optimizers import Adam
import keras.backend as K
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pydot
from IPython.display import SVG

Using TensorFlow backend.


## dataset

In [2]:
# and, or, xor, nor, nand
X = np.array([[0,0,0],[0,1,0],[1,0,0],[1,1,0], #and
              [0,0,1],[0,1,1],[1,0,1],[1,1,1], #or
              [0,0,2],[0,1,2],[1,0,2],[1,1,2], #xor
              [0,0,3],[0,1,3],[1,0,3],[1,1,3], #nor
              [0,0,4],[0,1,4],[1,0,4],[1,1,4]]) #nand
Y = np.array([0,0,0,1, #and
              0,1,1,1, #or
              0,1,1,0, #xor
              1,0,0,0, #nor
              1,1,1,0]) #nand

In [3]:
Y = keras.utils.to_categorical(Y, 2)

In [4]:
print(X.shape)
print(Y.shape)
print(X)
print(Y)

(20, 3)
(20, 2)
[[0 0 0]
 [0 1 0]
 [1 0 0]
 [1 1 0]
 [0 0 1]
 [0 1 1]
 [1 0 1]
 [1 1 1]
 [0 0 2]
 [0 1 2]
 [1 0 2]
 [1 1 2]
 [0 0 3]
 [0 1 3]
 [1 0 3]
 [1 1 3]
 [0 0 4]
 [0 1 4]
 [1 0 4]
 [1 1 4]]
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


## using Keras.backend.switch

In [5]:
# input layer
inputs = Input(shape=X.shape[1:],name="input")

In [6]:
# and layers and or layers
def andGate(inputs=inputs):
    andDense = Dense(32, activation="relu", name="and1")(inputs) 
    return andDense
def orGate(inputs=inputs):
    orDense = Dense(32, activation="sigmoid", name="or1")(inputs)     
    return orDense
def xorGate(inputs=inputs):
    xorDense = Dense(32, activation="tanh", name="xor1")(inputs)
    return xorDense
def norGate(inputs=inputs):
    norDense = Dense(32, activation="sigmoid", name="nor1")(inputs)
    norDense = Dense(32, activation="relu", name="nor2")(norDense)
    return norDense
def nandGate(inputs=inputs):
    nandDense = Dense(16, activation="relu", name="nand1")(inputs)
    nandDense = Dense(32, activation="relu", name="nand2")(nandDense)
    return nandDense

In [7]:
x_case = Lambda(lambda x: K.switch(K.equal(x[:,2],0),
                                   andGate(x[:,:2]),
                                   K.switch(K.equal(x[:,2],1),
                                            orGate(x[:,:2]),
                                            K.switch(K.equal(x[:,2],2),
                                                     xorGate(x[:,:2]),
                                                     K.switch(K.equal(x[:,2],3),
                                                              norGate(x[:,:2]),
                                                              nandGate(x[:,:2]))))),
                  output_shape=(32,))(inputs)

In [8]:
# output layer
outputs = Dense(2, activation="softmax", name="softmax")(x_case)
# model
model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 3)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 32)                0         
_________________________________________________________________
softmax (Dense)              (None, 2)                 66        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


In [9]:
# train and evaluate
model.fit(X, Y,
          batch_size=4,
          epochs=5000,
          verbose=0,
          shuffle=True)
model.evaluate(X,Y)

20/20 [==============================] - 0s 973us/step


[0.49733442068099976, 0.75]

## case function

In [10]:
def case(case_and_proc, default):
    """
    case_and_proc: pairs of case and process in list type or dict type
    default: default process. i.e. executed whenever the condition doesn't match any case
    """
    print(case_and_proc)
    if type(case_and_proc) == list:
        for cnum in range(len(case_and_proc)):
            if cnum == 0:
                print(case_and_proc[cnum][1])
                print(default)
                x = K.switch(case_and_proc[cnum][0], case_and_proc[cnum][1], default)
            else:
                print(case_and_proc[cnum][1])
                x = K.switch(case_and_proc[cnum][0], case_and_proc[cnum][1], x)
    elif type(case_and_proc) == dict:
        for i, (cond, expression) in enumerate(case_and_proc.items()):
            if i == 0:
                print(case_and_proc[cond])
                print(default)
                x = K.switch(cond, expression, default)
            else:
                print(case_and_proc[cond])
                x = K.switch(cond, expression, x)
    return x

In [11]:
# input layer
inputs = Input(shape=X.shape[1:],name="input")

In [12]:
x_case = Lambda(lambda x: case({K.equal(x[:,2],0): andGate(x[:,:2]),
                                K.equal(x[:,2],1): orGate(x[:,:2]),
                                K.equal(x[:,2],2): xorGate(x[:,:2]),
                                K.equal(x[:,2],3): norGate(x[:,:2])},
                               default=nandGate(x[:,:2])),
               output_shape=(32,))(inputs)

{<tf.Tensor 'lambda_2/Equal:0' shape=(?,) dtype=bool>: <tf.Tensor 'lambda_2/and1/Relu:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'lambda_2/Equal_1:0' shape=(?,) dtype=bool>: <tf.Tensor 'lambda_2/or1/Sigmoid:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'lambda_2/Equal_3:0' shape=(?,) dtype=bool>: <tf.Tensor 'lambda_2/nor2/Relu:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'lambda_2/Equal_2:0' shape=(?,) dtype=bool>: <tf.Tensor 'lambda_2/xor1/Tanh:0' shape=(?, 32) dtype=float32>}
Tensor("lambda_2/and1/Relu:0", shape=(?, 32), dtype=float32)
Tensor("lambda_2/nand2/Relu:0", shape=(?, 32), dtype=float32)
Tensor("lambda_2/or1/Sigmoid:0", shape=(?, 32), dtype=float32)
Tensor("lambda_2/nor2/Relu:0", shape=(?, 32), dtype=float32)
Tensor("lambda_2/xor1/Tanh:0", shape=(?, 32), dtype=float32)


In [13]:
# output layer
outputs = Dense(2, activation="softmax", name="softmax")(x_case)
# model
model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 3)                 0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 32)                0         
_________________________________________________________________
softmax (Dense)              (None, 2)                 66        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


In [14]:
# train and evaluate
model.fit(X, Y,
          batch_size=4,
          epochs=5000,
          verbose=0,
          shuffle=True)
model.evaluate(X,Y)

20/20 [==============================] - 0s 2ms/step


[0.38875028491020203, 0.85000002384185791]

In [15]:
# input layer
inputs = Input(shape=X.shape[1:],name="input")

In [16]:
x_case = Lambda(lambda x: case([(K.equal(x[:,2],0), andGate(x[:,:2])),
                                (K.equal(x[:,2],1), orGate(x[:,:2])),
                                (K.equal(x[:,2],2), xorGate(x[:,:2])),
                                (K.equal(x[:,2],3), norGate(x[:,:2]))],
                               default=nandGate(x[:,:2])),
               output_shape=(32,))(inputs)

[(<tf.Tensor 'lambda_3/Equal:0' shape=(?,) dtype=bool>, <tf.Tensor 'lambda_3/and1/Relu:0' shape=(?, 32) dtype=float32>), (<tf.Tensor 'lambda_3/Equal_1:0' shape=(?,) dtype=bool>, <tf.Tensor 'lambda_3/or1/Sigmoid:0' shape=(?, 32) dtype=float32>), (<tf.Tensor 'lambda_3/Equal_2:0' shape=(?,) dtype=bool>, <tf.Tensor 'lambda_3/xor1/Tanh:0' shape=(?, 32) dtype=float32>), (<tf.Tensor 'lambda_3/Equal_3:0' shape=(?,) dtype=bool>, <tf.Tensor 'lambda_3/nor2/Relu:0' shape=(?, 32) dtype=float32>)]
Tensor("lambda_3/and1/Relu:0", shape=(?, 32), dtype=float32)
Tensor("lambda_3/nand2/Relu:0", shape=(?, 32), dtype=float32)
Tensor("lambda_3/or1/Sigmoid:0", shape=(?, 32), dtype=float32)
Tensor("lambda_3/xor1/Tanh:0", shape=(?, 32), dtype=float32)
Tensor("lambda_3/nor2/Relu:0", shape=(?, 32), dtype=float32)


In [17]:
# output layer
outputs = Dense(2, activation="softmax", name="softmax")(x_case)
# model
model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 3)                 0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 32)                0         
_________________________________________________________________
softmax (Dense)              (None, 2)                 66        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


In [18]:
# train and evaluate
model.fit(X, Y,
          batch_size=4,
          epochs=5000,
          verbose=0,
          shuffle=True)
model.evaluate(X,Y)

20/20 [==============================] - 0s 2ms/step


[0.43193340301513672, 0.85000002384185791]